Goal → 후보 전이(s1↔s2) 중, THz 주파수 대역에 들고
        가시광(500–600 nm) 방출 캐스케이드 확률이 높은 쌍 찾기
        FoM = |Ω(1 V/m)| × visible_yield

[Inputs]
- ATOM_SPECIES, F_THz_MIN/MAX, N_MIN/MAX
- L_PAIRS, J_OPTIONS
- TEMP_K (예: 50 °C), visible window
- MC 파라미터 (shots, steps, BBR upward on/off)

[Nested loops]
(l1,l2) ∈ L_PAIRS →
  j1 ∈ J_OPTIONS[l1], j2 ∈ J_OPTIONS[l2] →
    n1 ∈ [N_MIN..N_MAX] →
      dn ∈ {0, ±1, ±2} → n2 = n1 + dn (in range)

[Freq filter]
f = |getTransitionFrequency(s1→s2)|
if f ∉ [F_THz_MIN..F_THz_MAX] → skip

[Coupling & dipole]
Ω(1 V/m) = |getRabiFrequency2(s1→s2)|
|d| = ħ·Ω / E (E=1 V/m)

[MC visible-yield @ TEMP_K]
- 그래프: E1 허용 이웃(Δl=±1)
- 하향: 자발 + BBR 유도 방출 / 상향: BBR 흡수(옵션)
- R = getTransitionRate(..., temperature=TEMP_K)
- 무작위 전이 선택(비례추출); 방출이면 λ가 500–600 nm에 있으면 카운트
- 충분히 낮은 상태 도달 또는 스텝 한도 시 종료
- y1=mc(s1,T), y2=mc(s2,T), visible_yield_best = max(y1,y2)

[Collect]
Append Candidate {s1,s2,f,|d|,Ω,yield}

[Rank]
정렬 by score = |Ω| × visible_yield_best → to_dataframe(topk)


In [1]:
# ✅ Drop-in fix (v2): temperature + Δn=0 + BBR upward(absorption) enabled
import math, time, random
from dataclasses import dataclass
from typing import List, Tuple
import pandas as pd
from arc import Caesium, Rubidium

# ----- user knobs -----
ATOM_SPECIES   = "Cs"                      # "Cs" or "Rb"
F_THz_MIN, F_THz_MAX = 0.1e12, 10e12      # [Hz] for candidate scanning only
N_MIN, N_MAX   = 6, 30
L_PAIRS        = [("D","S"), ("P","D"), ("F", "D")] # upper-lower pairs
J_OPTIONS      = {"S":[0.5], "P":[0.5, 1.5], "D":[1.5, 2.5], "F":[2.5, 3.5]} # valid j for each l
VISIBLE_WINDOW = (500e-9, 600e-9)          # [m]
ETHz_FOR_FOM   = 1.0                       # compare at 1 V/m
MC_CASCADES    = 800
MAX_DECAY_STEPS= 8
TOPK           = 100
SEED           = 7
random.seed(SEED)

# 🔹 Experiment temperature (e.g., 50 °C)
TEMP_K = 273.15 + 50.0

# 🔹 Upward transitions (BBR absorption) control & bounds
ALLOW_BBR_UP    = True     # include upward hops due to BBR
MC_N_FLOOR      = 5        # lowest n to consider in MC graph
MC_N_CEIL       = max(N_MAX, 40)  # highest n to consider (cap upward wandering)
MC_DOWN_SPAN    = 14       # per-hop max Δn downward range
MC_UP_SPAN      = 8        # per-hop max Δn upward range (tune for speed/accuracy)

HBAR = 1.054_571_817e-34
C    = 299_792_458.0

# polarization & mj (for getRabiFrequency2)
POL_Q = 0   # -1 (σ-), 0 (π), +1 (σ+)
M_J   = 0.5 # choose a valid mj for the chosen j1

def make_atom(name:str):
    return Caesium() if name.lower() in ("cs","caesium","cesium") else Rubidium()

def l_letter_to_l(ltr:str)->int:
    return {"S":0,"P":1,"D":2,"F":3,"G":4,"H":5}[ltr.upper()]

def trans_freq_hz(atom, s1, s2)->float:
    n1,l1,j1 = s1; n2,l2,j2 = s2
    return abs(atom.getTransitionFrequency(n1,l1,j1, n2,l2,j2))

def trans_wavelength_m(atom, s1, s2)->float:
    n1,l1,j1 = s1; n2,l2,j2 = s2
    return abs(atom.getTransitionWavelength(n1,l1,j1, n2,l2,j2))

def in_visible(lam:float, window=VISIBLE_WINDOW)->bool:
    return (lam>0) and (window[0] <= lam <= window[1])

# 🔹 helper: emission vs absorption 구분 (에너지 비교)
def is_emission(atom, s_from, s_to)->bool:
    # ARC getEnergy: energy (eV) relative to ionization (bound states are negative).
    # Emission if E_to < E_from (더 낮은 에너지로 내려감).
    E_from = atom.getEnergy(*s_from)
    E_to   = atom.getEnergy(*s_to)
    return E_to < E_from

# 🔹 E1 selection-rule 이웃(상·하향 모두) 생성
def e1_neighbors(n:int,l:int,j:float,
                 n_floor:int=MC_N_FLOOR,
                 n_ceiling:int=MC_N_CEIL,
                 down_span:int=MC_DOWN_SPAN,
                 up_span:int=MC_UP_SPAN,
                 include_up:bool=True,
                 include_down:bool=True):
    outs = []
    for lp in (l-1, l+1):
        if lp < 0: 
            continue
        for jp in (lp-0.5, lp+0.5):
            if jp <= 0: 
                continue
            if include_down and (n-1 >= n_floor):
                nmin = max(n_floor, n - down_span)
                for np in range(nmin, n):
                    outs.append((np,lp,jp))
            if include_up and (n+1 <= n_ceiling):
                nmax = min(n_ceiling, n + up_span)
                for np in range(n+1, nmax+1):
                    outs.append((np,lp,jp))
    # de-duplicate
    uniq, seen = [], set()
    for t in outs:
        if t not in seen:
            seen.add(t); uniq.append(t)
    return uniq

# --- Monte Carlo cascade (상·하향 포함; 온도 의존 전이율) ---
def mc_visible_yield(atom, n:int,l:int,j:float,
                     shots:int=MC_CASCADES,
                     max_steps:int=MAX_DECAY_STEPS,
                     window:Tuple[float,float]=VISIBLE_WINDOW,
                     temperature_K: float = 0.0,
                     allow_upward: bool = ALLOW_BBR_UP)->float:
    """
    Radiative cascade with temperature-dependent rates.
    - includes downward (spontaneous + BBR-stim.) and upward (BBR absorption) hops
    - counts ONLY emitted visible photons (absorption은 미집계)
    """
    vis = 0
    for _ in range(shots):
        cur = (n,l,j)
        for _ in range(max_steps):
            neigh = e1_neighbors(*cur, include_up=allow_upward, include_down=True)
            rate_list = []
            for t in neigh:
                try:
                    # ARC: temperature>0 이면 BBR 유도 전이 포함한 총 전이율 반환
                    R = atom.getTransitionRate(cur[0],cur[1],cur[2],
                                               t[0],t[1],t[2],
                                               temperature=temperature_K)
                except Exception:
                    R = 0.0
                if R > 0.0:
                    rate_list.append((t, R))
            if not rate_list:
                break
            totalR = sum(R for _,R in rate_list)
            r, acc = random.random()*totalR, 0.0
            next_state = None
            for t,R in rate_list:
                acc += R
                if r <= acc:
                    next_state = t; break
            # 가시광 "방출"만 카운트
            if next_state is None:
                break
            if is_emission(atom, cur, next_state):
                lam = trans_wavelength_m(atom, cur, next_state)
                if in_visible(lam, window):
                    vis += 1
            # 종료 조건: 충분히 낮은 상태로 떨어지면 중단 (S/P, n<=7 근처)
            if next_state[0] <= 7 and next_state[1] <= 1:
                break
            cur = next_state
    return vis/float(shots)

# --- Rabi (use ARC helper; no undefined local vars) ---
def omega_1Vpm(atom, s1, s2, mj1=M_J, q=POL_Q)->float:
    n1,l1,j1 = s1; n2,l2,j2 = s2
    try:
        return abs(atom.getRabiFrequency2(n1,l1,j1, mj1, n2,l2,j2, q, 1.0))  # rad/s @ 1 V/m
    except Exception:
        return 0.0

def d_from_omega(omega_rad_s:float, E=1.0)->float:
    return HBAR*omega_rad_s/E

@dataclass
class Candidate:
    upper: Tuple[int,int,float]
    lower: Tuple[int,int,float]
    f_THz: float
    d_Cm: float
    Omega_1Vpm: float
    vis_yield_best: float

def scan_candidates(atom_name="Cs")->List[Candidate]:
    atom = make_atom(atom_name)
    out: List[Candidate] = []
    t0 = time.time()
    for l1ltr, l2ltr in L_PAIRS:
        l1 = l_letter_to_l(l1ltr); l2 = l_letter_to_l(l2ltr)
        for j1 in J_OPTIONS[l1ltr]:
            for j2 in J_OPTIONS[l2ltr]:
                for n1 in range(N_MIN, N_MAX+1):
                    for dn in (0, -1, +1, -2, +2):
                        n2 = n1 + dn
                        if n2 < N_MIN or n2 > N_MAX:
                            continue

                        s1, s2 = (n1,l1,j1), (n2,l2,j2)

                        # --- NEW: 결과에 '상향'이 절대 안 나오게 보장 ---
                        if not is_emission(atom, s1, s2):
                            # 스왑했을 때 방출이면 방향을 바꿔서 채택
                            if is_emission(atom, s2, s1):
                                s1, s2 = s2, s1
                            else:
                                # 둘 다 방출쌍이 아니면 제외
                                continue
                        # -------------------------------------------------

                        f = trans_freq_hz(atom, s1, s2)
                        if F_THz_MIN <= f <= F_THz_MAX:
                            omega_rad_s = omega_1Vpm(atom, s1, s2)
                            d_cm = d_from_omega(omega_rad_s, E=ETHz_FOR_FOM)

                            # (선택) 상부 상태에서만 MC를 돌려도 됨:
                            # y1 = mc_visible_yield(atom, *s1, temperature_K=TEMP_K, allow_upward=ALLOW_BBR_UP)
                            # vis_best = y1
                            # 현재 로직 유지하려면 아래 두 줄 사용:
                            y1 = mc_visible_yield(atom, *s1, temperature_K=TEMP_K, allow_upward=ALLOW_BBR_UP)
                            #y2 = mc_visible_yield(atom, *s2, temperature_K=TEMP_K, allow_upward=ALLOW_BBR_UP)
                            vis_best = y1

                            out.append(Candidate(
                                upper=s1, lower=s2, f_THz=f,
                                d_Cm=d_cm, Omega_1Vpm=omega_rad_s,
                                vis_yield_best=vis_best
                            ))
    print(f"scan done: {len(out)} candidates in {time.time()-t0:.1f}s")
    return out

def score(c: Candidate)->float:
    return abs(c.Omega_1Vpm) * c.vis_yield_best

def fmt_state(s):
    n,l,j = s
    LTR = "SPDFGH"[l] if l<6 else f"L{l}"
    return f"{n}{LTR}_{int(2*j)}/2"

def to_dataframe(cands: List[Candidate], topk=TOPK)->pd.DataFrame:
    rows = []
    for c in sorted(cands, key=score, reverse=True)[:topk]:
        rows.append({
            "upper": fmt_state(c.upper),
            "lower": fmt_state(c.lower),
            "f_THz": c.f_THz/1e12,
            "|d| [C·m] (from Ω)": c.d_Cm,
            "Ω(1 V/m) [Hz]": c.Omega_1Vpm/(2*math.pi),
            "visible_yield(500–600nm)": c.vis_yield_best,
            "FoM = Ω×yield": score(c)
        })
    return pd.DataFrame(rows)


In [ ]:
cands = scan_candidates(ATOM_SPECIES)  # <-- 더 이상 UnboundLocalError 안 납니다
df = to_dataframe(cands, topk=TOPK)
df

In [ ]:
df.to_csv("CS_0.1-10THz-SD to PDF-candidates.csv", index=False, encoding="utf-8")

print("CSV 파일 저장 완료: CS_0.1-10THz-SDF to P-candidates.csv")

CSV 파일 저장 완료: candidates.csv
